In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append('%s/../' % os.getcwd())

import datetime
import pandas as pd
import numpy as np
import getpass

In [2]:
from formulas_auxiliares_ajuste_base import months_dict, dict_contas
import formulas_auxiliares_ajuste_base as aux
from class_ajusta_batimento_email import *

In [3]:
path = '\\\\DRIOC0231PFS\\Apoio_SB\\Geral\\BD\\Aluguel_Offshore\\Arquivos diários\\jul2018\\consolidado\\'
file = 'bases_batimento_pb_calypso.xlsx'

In [4]:
pasta = r'C:\Users\correigu\OneDrive - Banco BTG Pactual S.A\Desktop\aqruivos_offshore'
data_inicio = pd.to_datetime('2018-07-01')
data_final = pd.to_datetime('2018-07-15')

In [5]:
abertura = pd.read_excel(path+'distribuicao_asset_consolidada_ajustada_email_cayman.xlsx', sheetname='tickers_abertura')
ajuste = pd.read_excel(path+'distribuicao_asset_consolidada_ajustada_email_cayman.xlsx', sheetname='ajuste')
abertura_batidos_consol = pd.read_excel(path+'distribuicao_asset_consolidada_ajustada_email_cayman.xlsx', 
                                        sheetname='trades_abertura_batidos_consol')

In [6]:
pb_wm = abertura[(abertura['book'].str.contains('PRIVATE'))\
                 |(abertura['book'].str.contains('CLIENT'))].copy()

In [7]:
ubs, morgan, intc, bdwm, pras = aux.TratamentoDosDados(pasta, data_inicio, data_final).baixa_dados_df()

Povoando lista de arquivos que serão baixados para a memória
    O arquivo Ajustes não possui data registrada
    O arquivo Ajustes não pôde ser comparado com os demais
    O arquivo tickers_diffs.xlsx não possui data registrada
    O arquivo tickers_diffs.xlsx não pôde ser comparado com os demais
--------------------------------------------------------------------------------------------------------------
Baixando arquivos!!
    10 arquivos do ubs importados
    20 arquivos da morgan importados
    10 arquivos do Calypso PROP+ASSET importados
    10 arquivos do Calypso BD+WM importados
    10 arquivos do Calypso INTC importados
--------------------------------------------------------------------------------------------------------------
Baixando arquivos!!
    10 arquivos do ubs importados
    20 arquivos da morgan importados
    10 arquivos do Calypso PROP+ASSET importados
    10 arquivos do Calypso BD+WM importados
    10 arquivos do Calypso INTC importados
-------------------------

In [8]:
calypso_wm = bdwm.copy()
calypso_wm['tipo'] = 'WM' 
calypso_wm['date'] = pd.to_datetime(calypso_wm['date'])
calypso_wm['month_number'] = calypso_wm['date'].dt.month
calypso_wm['year'] = calypso_wm['date'].dt.year
calypso_wm['month'] = calypso_wm['month_number'].map(months_dict)
calypso_wm = calypso_wm[['Account Name','tipo', 'Agent','date', 'shares', 'Book', 'PRODUCT_CODE.TICKER', 'PRODUCT_CODE.ISIN', 
                   'PRODUCT_CODE.CUSIP', 'Product Currency', 'month', 'year', 'LE Name']].reset_index(drop=True)
calypso_wm.columns = ['conta_pb','conta', 'agent', 'data_ref', 'quantidade', 'book', 'ticker', 'isin', 'cusip', 
              'currency', 'month', 'year', 'cliente']
calypso_wm['quantidade'] = calypso_wm['quantidade'].replace(',','', regex=True)
calypso_wm['quantidade'] = calypso_wm['quantidade'].astype(float)

In [9]:
calypso_wm['quantidade'] = -1*calypso_wm['quantidade']

In [10]:
calypso_wm = calypso_wm[calypso_wm['quantidade']<0]

In [11]:
calypso_wm_grouped =calypso_wm.groupby(by=['data_ref', 'book', 'ticker', 'currency']).sum().reset_index()

In [12]:
check_17 = pb_wm.shape[0]-pd.merge(pb_wm, calypso_wm_grouped, on=['data_ref', 'book', 'ticker', 'currency'], how='left')\
                                                                                    .shape[0]
if check_17!=0:
    print ('%s linhas foram duplicadas durante o barimento de WM!!!! Verificar!' %check_17)

In [13]:
batimento_wm = pd.merge(pb_wm, calypso_wm_grouped, on=['data_ref', 'book', 'ticker', 'currency'], how='left')\
                                                                                    .drop('year', axis=1)

In [14]:
batimento_wm_sobra_destino = batimento_wm[batimento_wm['quantidade'].isnull()==True].copy()
print ('Durante o batimento de WM, %s trades ficaram como sobra de destino!!' %batimento_wm_sobra_destino.shape[0])

batimento_wm_sobra_destino['status'] = 'n_batido'

Durante o batimento de WM, 0 trades ficaram como sobra de destino!!


In [15]:
batimento_wm = batimento_wm[batimento_wm['quantidade'].isnull()==False].copy()
batimento_wm_batidos = batimento_wm[batimento_wm['quantidade_book']==batimento_wm['quantidade']].copy()

batimento_wm_batidos['status'] = 'batido'

In [16]:
batimento_wm_divergentes = batimento_wm[batimento_wm['quantidade_book']!=batimento_wm['quantidade']].copy()
print ('Durante o batimento de WM, %s trades ficaram com quantidade divergente!!' %batimento_wm_divergentes.shape[0])

Durante o batimento de WM, 1 trades ficaram com quantidade divergente!!


In [17]:
batimento_wm_divergentes['status'] = 'divergente'

In [18]:
batimento_wm = pd.concat([batimento_wm_batidos, batimento_wm_divergentes], axis=0)

In [19]:
batimento_wm['id'] = range(0, batimento_wm.shape[0])

In [20]:
batimento_wm_distrib = pd.merge(calypso_wm, batimento_wm, on=['data_ref', 'book', 'ticker', 'currency'], how='left')\
[['conta_pb_x', 'agent_x', 'data_ref', 'quantidade_x', 'book', 'ticker', 'currency', 'cliente','quantidade_book', 
  'acc', 'status','accrual_final', 'id']].rename(columns={
    'quantidade_book':'quantidade_ticker',
    'quantidade_x':'quantidade_book',
    'conta_pb_x':'conta_pb',
    'agent_x':'agent',
    'cliente':'conta'   
})
check_18 = batimento_wm_distrib.shape[0]-calypso_wm.shape[0]
if check_18!=0:
    print ('%s linhas foram duplicadas durante o ajuste de WM' %check_18)

In [21]:
batimento_wm_sobra_origem = batimento_wm[~batimento_wm_distrib['id'].isin(batimento_wm['id'].copy().drop_duplicates())].copy()
print ('Durante o batimento de WM, %s trades não foram distribuídos!!' %batimento_wm_sobra_origem.shape[0])

Durante o batimento de WM, 54 trades não foram distribuídos!!


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [22]:
batimento_wm_distrib = batimento_wm_distrib.drop('id', axis=1)

In [23]:
n_batimento_wm_distrib = batimento_wm_distrib[batimento_wm_distrib['quantidade_ticker'].isnull()==True]\
                                                                .drop(['quantidade_ticker'], axis=1)

In [24]:
batimento_wm_distrib = batimento_wm_distrib[batimento_wm_distrib['quantidade_ticker'].isnull()==False]

In [25]:
batimento_wm_distrib['pct'] = batimento_wm_distrib['quantidade_book']/batimento_wm_distrib['quantidade_ticker']

In [26]:
batimento_wm_distrib['accrual'] = batimento_wm_distrib['accrual_final']*batimento_wm_distrib['pct']

In [27]:
batimento_wm_distrib = batimento_wm_distrib.drop(['quantidade_ticker', 'pct'], axis=1).rename(columns = {
'accrual_final':'accrual_n_distrib','accrual':'accrual_final'})

In [28]:
abertura_batidos_consol_wm = pd.merge(batimento_wm_distrib, ajuste, on=['acc','currency'], how='left')

In [29]:
abertura_batidos_consol_wm['accrual'] = abertura_batidos_consol_wm['accrual_final']*abertura_batidos_consol_wm['multiplicador']

In [30]:
abertura_batidos_consol_wm = abertura_batidos_consol_wm.drop(['accrual_final', 'multiplicador','accrual_n_distrib'], axis=1)

In [31]:
abertura_batidos_consol_wm = abertura_batidos_consol_wm[abertura_batidos_consol_wm['accrual']!=0]

In [32]:
total_wm = abertura_batidos_consol_wm.groupby(by=['acc','book','currency']).sum().reset_index()

In [33]:
# Aqui eu preciso fazer um pequeno ajuste pra parar de considerar a aberta do books CLIENT e distribuir o que sobra pros clients
# que já foram distribuidos!!!

In [34]:
total_wm_calypso = abertura_batidos_consol[(abertura_batidos_consol['book'].str.contains('CLIENTS'))|\
                       (abertura_batidos_consol['book'].str.contains('PRIVATE'))].reset_index(drop=True)\
                                                                .groupby(by=['book', 'currency']).sum().reset_index()

In [35]:
total_wm_n_distrib = pd.merge(total_wm, total_wm_calypso, on=['book','currency'], how='left').fillna(0)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\tools\merge.py:1376: RuntimeWarning: divide by zero encountered in longlong_scalars
  stride //= shape[i]


In [36]:
total_wm_n_distrib['accrual_final_n_distrib'] = total_wm_n_distrib['accrual_final']-total_wm_n_distrib['accrual']

In [37]:
total_wm_n_distrib = total_wm_n_distrib.drop(['accrual', 'accrual_final', 'quantidade_book_x','quantidade_book_y',
                                              'conta_pb'], axis=1)

In [38]:
abertura_batidos_consol_wm = abertura_batidos_consol_wm.groupby(by=['book','conta','currency']).sum().reset_index()\
                                                .drop(['conta_pb','quantidade_book'], axis=1)

In [39]:
total_wm_n_distrib['conta'] = 'OUTROS'
total_wm_n_distrib = total_wm_n_distrib.rename(columns={'accrual_final_n_distrib':'accrual'})

In [40]:
abertura_batidos_consol_wm = abertura_batidos_consol_wm.groupby(by=['conta', 'book','currency']).sum().reset_index()
abertura_batidos_consol_wm['acc'] = 'ubs_cayman'

In [41]:
abertura_batidos_consol_wm = pd.concat([total_wm_n_distrib, abertura_batidos_consol_wm], axis=0)

In [42]:
abertura_batidos_consol_wm = abertura_batidos_consol_wm[['acc','conta','book','currency','accrual']]

In [43]:
abertura_batidos_consol = abertura_batidos_consol[['acc', 'book', 'currency', 'accrual_final']]